In [87]:
import os
import json 
import pandas as pd 
import traceback

In [88]:
from langchain.chat_models import ChatOpenAI

In [89]:
from langchain_openai import ChatOpenAI

In [90]:
KEY=os.getenv('OPENAI_API_API')

In [91]:
llm = ChatOpenAI(openai_api_key=KEY,model_name = "gpt-3.5-turbo")

In [92]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # Load .env file


True

In [93]:
KEY=os.getenv('OPENAI_API_KEY')

In [94]:
from dotenv import load_dotenv
load_dotenv()

True

In [95]:
print(KEY)

"sk-proj-vGYb2GIjM86mxx0je9xwho8CJH6pqCd1diUnA6Ui


In [96]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021530F7F680>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021530DB0B90>, root_client=<openai.OpenAI object at 0x000002152DB45C40>, root_async_client=<openai.AsyncOpenAI object at 0x0000021530F7D310>, model_kwargs={})

In [97]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain

import PyPDF2



In [98]:
"it is the format of the quiz"

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice quesrion",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice quesrion",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice quesrion",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    
}

In [99]:
"it is a template for the generation of the quiz. the whole quiz qns are generated based on the below template"
"this is what we pass to our gpt model"

TEMPLATE="""Text:{text}
    You are an MCQ maker. Given the above text, it is your job to \
    create a quiz of {number} multiple choice questions for {subject} studenrs is {tone} tone.
    make sure the question are not repeated and check all the questions to be conforming the text as well.
    Make sure tto format your response like RESPONSE_JSON below nd use it as a guide. \
    Ensure to make {number} MCQs
    ### RESPONSE_JSON
    {response_json}
    
    """

In [100]:
"here we define the prompt template"

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

"the input variables are from the user side"

'the input variables are from the user side'

In [101]:
"it is a chain to to combine the llm and the prompt"

quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [102]:
"""after analyzing the quiz for the prompt we need the correct answer.
 for this i have created another template callled templete2"""

TEMPLATE2="""
You are an expert English grammarian and writer given at multiple choice quiz per {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz.
Only use at maximum of 50 words per quiz. 
The quiz is not per with the cognitive and analytical abilities of the student,\
Compare to quiz, questions which need to be changed and change the tool such that it perfectly fits the student abilities. 
Quiz_MCQ:
{quiz}

check for an expert English writer of the above quiz:
"""

In [103]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [104]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [105]:
"now we have two chains here , we combine those two chain with the help of the sequential chain method."


generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number", "subject","tone", "response_json"], 
                                        output_variables=["quiz", "review"], verbose=True,)

In [106]:
"we need the data for generating the quiz. so we created a data.txt file in which i had entered the ai wikipedia data"

file_path = r"C:\Users\MOHAMMED FEROZ\mcqgenerator\data.txt"

"""r before the path is indicates the absolute path """

'r before the path is indicates the absolute path '

In [107]:
file_path

'C:\\Users\\MOHAMMED FEROZ\\mcqgenerator\\data.txt'

In [108]:
"to open and read the file"

with open(file_path, 'r') as file:
    TEXTS = file.read()

In [109]:
print(TEXTS)

AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).
Part of a series on
Artificial intelligence (AI)


Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and an

In [110]:
"in the above we given the format of the quiz which is in the dictionary format. now we convert it into the json format"

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice quesrion", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice quesrion", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice quesrion", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [111]:
NUMBER = 5
SUBJECT = "artificial intelligence"
TONE = "simple"

In [116]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain({
        "text": TEXTS,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)

    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Text:AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).
Part of a series on
Artificial intelligence (AI)


Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous v

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-proj*************************************A6Ui. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [117]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Ptompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Ptompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [ ]:
"the token values will be shown once we give the premium OPenAI API"

'the token values will be shown once we give the premium OPenAI API'

In [ ]:
response

"not running because of the API access"

NameError: name 'response' is not defined

In [120]:
quiz=response.get("quiz")

"the error is because of the API access"

NameError: name 'response' is not defined

In [122]:
json.loads(quiz)

NameError: name 'quiz' is not defined

In [123]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".json(
        [
        f"{option}: {option_value}"
        for option, option_value in value["options"].items()
        
        ]
    )
    correct = value["correct"]
    quiz_tabke_data.append({"MCQ":mcq, "Choices": options, "Correct": correct})

NameError: name 'quiz' is not defined

In [ ]:
quiz.pd.DataFrame(quiz_table_data)

In [1]:
quiz.to_csv("artificial intelligence", index=False)

NameError: name 'quiz' is not defined

In [6]:
import logging 
import os
from datetime import datetime

LOF_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

print(LOF_FILE)

05_23_2025_10_47_14.log
